In [1]:
import pandas as pd
df=messages = pd.read_csv('SMSSpamCollection', sep='\t', names=["label", "message"])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
df.shape

(5572, 2)

In [3]:
X=list(df['message'])
print(len(X))
y=list(df['label'])
print(len(y))

5572
5572


In [4]:
y=list(pd.get_dummies(y,drop_first=True)['spam'])

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [6]:
y_train = pd.Series(y_train).astype(int)
y_test = pd.Series(y_test).astype(int)

In [7]:
!pip install transformers==4.18.0

In [8]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [9]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [10]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))


In [14]:
train_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(238,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(238,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [15]:
test_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(221,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(221,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [16]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

In [30]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                        
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=test_dataset            
)

trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
trainer.predict(test_dataset)

In [ ]:
output=trainer.predict(test_dataset)[1]


In [ ]:
trainer.save_model('sentiment_model')